In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2157403/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
2,model_9_9_10,0.903285,0.831456,0.824307,0.829321,0.107035,0.294544,0.147079,0.225148,"Hidden Size=[15], regularizer=0.02, learning_r..."
3,model_9_9_9,0.903198,0.828178,0.832367,0.829429,0.107131,0.300272,0.140331,0.225006,"Hidden Size=[15], regularizer=0.02, learning_r..."
5,model_9_7_22,0.902361,0.721901,0.933477,0.868777,0.108058,0.265952,0.162403,0.217223,"Hidden Size=[15], regularizer=0.02, learning_r..."
6,model_9_7_21,0.902330,0.721207,0.933765,0.868765,0.108092,0.266615,0.161699,0.217243,"Hidden Size=[15], regularizer=0.02, learning_r..."
7,model_9_7_20,0.902242,0.719617,0.934227,0.868600,0.108190,0.268137,0.160570,0.217517,"Hidden Size=[15], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
256,model_9_5_1,0.851560,0.770616,0.595144,0.704406,0.164280,0.239640,0.395967,0.313206,"Hidden Size=[15], regularizer=0.02, learning_r..."
290,model_11_1_1,0.830743,0.854374,0.613938,0.714828,0.187318,0.172802,0.714889,0.427902,"Hidden Size=[15], regularizer=0.1, learning_ra..."
294,model_6_0_1,0.826704,0.424919,0.883021,0.808407,0.191788,0.221135,0.248841,0.234173,"Hidden Size=[10], regularizer=0.02, learning_r..."
299,model_9_1_1,0.825631,0.591670,0.860928,0.689944,0.192975,0.447168,0.091533,0.279810,"Hidden Size=[15], regularizer=0.02, learning_r..."
